# KWood - SVS Carbon Emissions

Polar plot of data, daily, overlayed on ice concentration

Data is in hdf format

In [95]:
import xarray as xr

import numpy as np
import h5py # if this creates an error please make sure you have the h5py library

import matplotlib.pyplot as plt
import matplotlib.path as mpath
from cartopy import config
import cartopy
import cartopy.crs as ccrs
import cmocean as cm

In [237]:
filein = ['/Users/bell/Downloads/GFED4.1s_2017_beta.hdf5','/Users/bell/Downloads/GFED4.1s_2018_beta.hdf5',
          '/Users/bell/Downloads/GFED4.1s_2019_beta.hdf5','/Users/bell/Downloads/GFED4.1s_2020_beta.hdf5']

In [243]:
count=0
sum_10_day = np.zeros((720, 1440))
for ofile in filein:
    f = h5py.File(ofile, 'r')
    for mm in range(1,12):
        month='/emissions/'+str(mm).zfill(2)+'/C/' 
        for dd in range(1,31):
            day='/emissions/'+str(mm).zfill(2)+'/daily_fraction/day_'+str(dd)+'/' 

            try:
                var = f[month][:,:] * f[day][:,:]
            except:
                continue

            if count < 9:
                sum_10_day = sum_10_day+var
                count+=1

            elif count == 9:
                sum_10_day = sum_10_day+var
                sum_10_day[sum_10_day==0]=np.nan

                #resource had 10day averages
                fig = plt.figure(figsize=[20, 20])
                ax = plt.axes(projection=ccrs.Orthographic(180, 90))


                #ax.set_extent([-180, 180, 45, 90], ccrs.PlateCarree())

                theta = np.linspace(0, 2*np.pi, 100)
                center, radius = [0.5, 0.5], 0.5
                verts = np.vstack([np.sin(theta), np.cos(theta)]).T
                circle = mpath.Path(verts * radius + center)

                ax.set_boundary(circle, transform=ax.transAxes)

                ax.pcolormesh(f['lon'],f['lat'],sum_10_day,vmin=0,vmax=30,
                             transform=ccrs.PlateCarree())

                ax.coastlines(resolution='50m')
                fig.savefig('CO_emmisions_daily'+ofile.split('.')[-2]+str(mm).zfill(2)+'_'+str(dd).zfill(2)+'_10day.png')
                plt.close(fig)

                sum_10_day = np.zeros((720, 1440))
                count=0
